# SAMI wind paper plots (multi-component fits)
---

In [1]:
%matplotlib widget

In [2]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist
from tqdm import tqdm
from scipy import constants

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, whav_colors, whav_labels
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.core.debugger import Tracer

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 14})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [3]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/paper/"
savefigs = True
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 5       # Minimum S/N of emission lines to accept
plt.close("all")


In [4]:
# Load the sample
df = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type=bin_type,
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        line_amplitude_SNR_cut=True,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc,

## PLOT: BPT diagram coloured by count, median number of components, BPT classification
---

In [7]:
for col_z in ["count"]: #, "Number of components", "BPT (numeric) (total)"]:
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)
    
    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, vmin=1 if col_z == "count" else None, vmax=1e3 if col_z == "count" else None, ymin=-1.2, ymax=1.6, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, vmin=1 if col_z == "count" else None, vmax=1e3 if col_z == "count" else None, ymin=-1.2, ymax=1.6, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, vmin=1 if col_z == "count" else None, vmax=1e3 if col_z == "count" else None, ymin=-1.2, ymax=1.6, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]
    
    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_count.pdf


## PLOT: BPT diagram coloured by $\sigma_{\rm gas}$ and by $\Delta \sigma$ for each component
---

In [118]:
plt.close("all")

In [119]:
col_z = "log sigma_gas"
for ii in range(3):
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {ii})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {ii})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {ii})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_component{ii + 1}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")

col_z = "sigma_gas - sigma_*"
for ii in range(3):
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {ii})", log_z=False, vmin=-200, vmax=+200, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {ii})", log_z=False, vmin=-200, vmax=+200, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {ii})", log_z=False, vmin=-200, vmax=+200, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_component{ii + 1}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_log_sigma_gas_component1.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_log_sigma_gas_component2.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_sigma_gas-sigma_star_component1.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_sigma_gas-sigma_star_component2.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/BPT_SAMI_sigma_gas-sigma_star_component3.pdf


## PLOT: bar chart showing the fraction of spaxels w/ 1,2,3 components in each spectral category
---

In [8]:
###############################################################################
# Stacked bar chart: # of fitted components by spectral classification
###############################################################################
counts_0 = []
counts_1 = []
counts_2 = []
counts_3 = []
for bpt in bpt_labels:
    cond = df["BPT (total)"] == bpt
    cond_0 = cond & (df["Number of components"] == 0)
    cond_1 = cond & (df["Number of components"] == 1) & ~df["Low S/N component - HALPHA (component 0)"]
    cond_2 = cond & (df["Number of components"] == 2) & ~df["Low S/N component - HALPHA (component 0)"] & ~df["Low S/N component - HALPHA (component 1)"]
    cond_3 = cond & (df["Number of components"] == 3) & ~df["Low S/N component - HALPHA (component 0)"] & ~df["Low S/N component - HALPHA (component 1)"] & ~df["Low S/N component - HALPHA (component 2)"]
    counts_0.append(df[cond_0].shape[0] / df[cond].shape[0] * 100)
    counts_1.append(df[cond_1].shape[0] / df[cond].shape[0] * 100)
    counts_2.append(df[cond_2].shape[0] / df[cond].shape[0] * 100)
    counts_3.append(df[cond_3].shape[0] / df[cond].shape[0] * 100)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5))
ax.bar(bpt_labels, counts_0, width=0.6, label=ncomponents_labels[0] + " components", color=ncomponents_colours[0])
ax.bar(bpt_labels, counts_1, width=0.6, bottom=counts_0, label=ncomponents_labels[1] + " component", color=ncomponents_colours[1])
ax.bar(bpt_labels, counts_2, width=0.6, bottom=np.array(counts_0) + np.array(counts_1), label=ncomponents_labels[2] + " components", color=ncomponents_colours[2])
ax.bar(bpt_labels, counts_3, width=0.6, bottom=np.array(counts_0) + np.array(counts_1) + np.array(counts_2), label=ncomponents_labels[3] + " components", color=ncomponents_colours[3])
ax.grid()
ax.set_ylim([0, 105])
ax.set_ylabel("Percentage")
ax.set_xlabel("Spectral classification")
ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.02), ncol=2, fontsize="small")
plt.xticks(rotation=90)

if savefigs:
    fname = os.path.join(fig_path, f"bar_ncomponents_by_spectral_class.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"File saved at: {fname}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/bar_ncomponents_by_spectral_class.pdf


## PLOT: WHAV diagram for the whole sample coloured by count, BPT category, $v_{\rm grad}$, $r/R_e$
---

In [9]:
col_z_list = ["count", "BPT (numeric) (total)", "v_grad", "r/R_e"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df, col_x=f"log sigma_gas (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_nostar_SAMI_{fname_fn(col_z)}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")        
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_nostar_SAMI_count.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_nostar_SAMI_BPT.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_nostar_SAMI_v_grad.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_nostar_SAMI_r_over_Re.pdf


## PLOT: WHAV* diagram for the whole sample coloured by count, BPT category
---

In [10]:
col_z_list = ["count", "BPT (numeric) (total)", "v_grad", "r/R_e"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_{fname_fn(col_z)}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")        
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_count.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_BPT.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_v_grad.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_r_over_Re.pdf


## PLOT: WHAV* diagram for each spectral classification separately
---

In [ ]:
col_z = "count"
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"

fig, axs_all = plt.subplots(nrows=len(bpt_labels[1:]), ncols=3, figsize=(5 * 3, 5 * len(bpt_labels[1:])), sharex=True)
fig.subplots_adjust(wspace=0, hspace=0)

for rr, (axs, bpt) in enumerate(zip(axs_all, bpt_labels[1:])): 
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.03, bbox.height])

    df_subset = df[df["BPT (total)"] == bpt]
    # recom components
    for ii in range(3):
        plot2dhistcontours(df_subset, col_x=f"{col_x} (component {ii})",
                           col_y=f"{col_y} (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].get_children()[0].set_clim([1, 1e3]) if col_z == "count" else None
        axs[ii].grid()
        axs[ii].axvline(0, linestyle="--", color="k") if col_x == "sigma_gas - sigma_*" else None
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].set_xlabel("") if rr < len(bpt_labels[1:]) - 2 else None
        axs[ii].set_xticklabels([]) if rr < len(bpt_labels[1:]) - 2 else None
        axs[ii].set_title(component_labels[ii]) if rr == 0 else None
    axs[0].text(s=bpt, x=0.1, y=0.9, horizontalalignment="left", transform=axs[0].transAxes)


## PLOT: BPT, WHAN, WHAV* for individual galaxies
---

In [134]:
plt.close("all")

In [135]:
# gal = 9011900521  # cool barred galaxy
gal = 209807
df_gal = df[df["ID"] == gal]
savefigs = True

In [136]:
# layout:
# SDSS image | number of components | BT classifications
# delta sigma: 1 | 2 | 3
# EW: 1 | 2 | 3
# BPT
# WHAN | WHAV

In [137]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(13, 17))
fig.subplots_adjust(wspace=0.05, hspace=0.05)

# SDSS image 
ax = plot_sdss_image(df_gal, ax=axs[0][0])
if ax is not None:
    ax.set_title(f"GAMA{gal}")
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

# Number of components
_, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="Number of components",
          ax=axs[0][1], 
          plot_colorbar=True, cax=None, cax_orientation="horizontal", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)


# BPT classifications 
_, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="BPT (numeric) (total)",
          ax=axs[0][2], 
          plot_colorbar=True, cax=None, cax_orientation="vertical", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)

# v_gas
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"v_gas (component {ii})",
              ax=axs[1][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=-200, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[1][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)


# delta sigma 
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"sigma_gas - sigma_* (component {ii})",
              ax=axs[2][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=-200, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[1][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

# EW 
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"HALPHA EW (component {ii})",
              ax=axs[3][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[2][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
        
# Save
if savefigs:
    fname = os.path.join(fig_path, f"GAMA{gal}_maps.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"Saving to {fname}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal["x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))


Saving to /priv/meggs3/u5708159/SAMI/figs/paper/GAMA209807_maps.pdf


In [138]:
fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True, figsize=(12, 4))
markers = ["o", ">", "D"]
col_z = "r/R_e"
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Plot full SAMI sample
    plot2dhistcontours(df=df, 
                       col_x=f"{col_x} (total)",
                       col_y=f"log O3 (total)", col_z="count", log_z=True,
                       alpha=0.5, cmap="gray_r",
                       ax=axs[cc], plot_colorbar=False)

    # Plot measurements for this galaxy
    plot2dscatter(df=df_gal,
                  col_x=f"{col_x} (total)",
                  col_y=f"log O3 (total)",
                  col_z=col_z,
                  cax=cax,
                  marker="o", ax=axs[cc], 
                  markersize=20, 
                  markerfacecolor=component_colours[ii],
                  markeredgecolor="black",
                  plot_colorbar=True if cc == 2 else False)

    # axis limits
    axs[cc].set_xlim([np.nanmin([vmin_fn(col_x), df_gal[f"{col_x} (total)"].min() - 0.1]),
                      np.nanmax([vmax_fn(col_x), df_gal[f"{col_x} (total)"].max() + 0.1])])
    # axis limits
    axs[cc].set_ylim([np.nanmin([vmin_fn("log O3"), df_gal[f"log O3 (total)"].min() - 0.1]),
                      np.nanmax([vmax_fn("log O3"), df_gal[f"log O3 (total)"].max() + 0.1])])

# Decorations
[ax.grid() for ax in axs]
[ax.set_ylabel("") for ax in axs[1:]]
[ax.set_yticklabels([]) for ax in axs[1:]]
[ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs[:-1]]
[ax.collections[0].set_rasterized(True) for ax in axs]

# Save
if savefigs:
    fname = os.path.join(fig_path, f"GAMA{gal}_BPT.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"Saving to {fname}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Saving to /priv/meggs3/u5708159/SAMI/figs/paper/GAMA209807_BPT.pdf


In [139]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.3)
col_z = "r/R_e"
col_y = "log HALPHA EW"


# WHAN diagram
ax = axs[0]

# Plot full SAMI sample
plot2dhistcontours(df=df, col_x=f"log N2 (total)",
                   col_y=f"log HALPHA EW (total)",
                   col_z="count", log_z=True,
                   alpha=0.5, cmap="gray_r", ax=ax,
                   plot_colorbar=False)

# Plot measurements for this galaxy
plot2dscatter(df=df_gal,
              col_x=f"log N2 (total)",
              col_y=f"log HALPHA EW (total)",
              col_z=col_z,
              cax=cax,
              marker="o", ax=ax, 
              markersize=20, 
              markeredgecolor="black",
              plot_colorbar=True, )

# axis limits
ax.set_xlim([np.nanmin([vmin_fn(f"log N2 (total)"), df_gal[f"log N2 (total)"].min() - 0.1]),
             np.nanmax([vmax_fn(f"log N2 (total)"), df_gal[f"log N2 (total)"].max() + 0.1])])
# axis limits
ax.set_ylim([np.nanmin([vmin_fn("log HALPHA EW"), df_gal[f"log HALPHA EW (total)"].min() - 0.1]),
             np.nanmax([vmax_fn("log HALPHA EW"), df_gal[f"log HALPHA EW (total)"].max() + 0.1])])

# Decorations
ax.grid()

# WHAV* diagram
ax = axs[1]

# Plot full SAMI sample
plot2dhistcontours(df=df, col_x=f"sigma_gas - sigma_*",
                   col_y=f"log HALPHA EW",
                   col_z="count", log_z=True,
                   alpha=0.5, cmap="gray_r", ax=ax,
                   plot_colorbar=False)

# Plot measurements for this galaxy
markers = ["o", ">", "D"]
for ii in range(3):
    plot2dscatter(df=df_gal,
                  col_x=f"sigma_gas - sigma_* (component {ii})",
                  col_y=f"log HALPHA EW (component {ii})",
                  marker=markers[ii], ax=ax, 
                  cax=None,
                  markersize=20, 
                  markerfacecolor=component_colours[ii], 
                  markeredgecolor="black",
                  plot_colorbar=False)

ax.set_xlim(
    [np.nanmin([vmin_fn("sigma_gas - sigma_*"), 
                df_gal["sigma_gas - sigma_* (component 0)"].min(), 
                df_gal["sigma_gas - sigma_* (component 1)"].min(), 
                df_gal["sigma_gas - sigma_* (component 2)"].min()]) - 50,
     np.nanmax([vmax_fn("sigma_gas - sigma_*"),
                df_gal["sigma_gas - sigma_* (component 0)"].max(),
                df_gal["sigma_gas - sigma_* (component 1)"].max(),
                df_gal["sigma_gas - sigma_* (component 2)"].max()]) + 50])

# Decorations
ax.grid()

# Legend
legend_elements = [Line2D([0], [0], marker=markers[ii], 
                          color="none", markeredgecolor="black",
                          label=f"Component {ii + 1}",
                          markerfacecolor=component_colours[ii], markersize=5) for ii in range(3)]
ax.legend(handles=legend_elements, fontsize="x-small", loc="upper right")

# Save
if savefigs:
    fname = os.path.join(fig_path, f"GAMA{gal}_WHAN_WHAV.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"Saving to {fname}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Saving to /priv/meggs3/u5708159/SAMI/figs/paper/GAMA209807_WHAN_WHAV.pdf


# **Analysis of star-forming spaxels**
---
---

In [13]:
plt.close("all")

In [5]:
# Make a separate DataFrame only containing star-forming spaxels
df_SF = df.copy()
df_SF = df_SF[df_SF["BPT (total)"] == "SF"]

# Inclination cut?
# df_SF = df_SF[df_SF["Inclination i (degrees)"] < 30]

# Remove spaxels at small radii?

# # Remove spaxels with v_grad > some cutoff?
# df_SF = df_SF[(df_SF["v_grad (component 0)"] < 10) &
#               ((df_SF["v_grad (component 1)"] < 10) | (df_SF["v_grad (component 1)"].isna())) & 
#               ((df_SF["v_grad (component 2)"] < 10) | (df_SF["v_grad (component 2)"].isna()))]

# df_SF = df_SF[df_SF["log N2 (total)"] <= -0.35]

### PLOT: histograms showing local/global/systematic quantities for 1,2,3-component spaxels: what drives the number of components per spaxel?
---

In [141]:
# Halpha EW, SFR, SFR surface density
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["log HALPHA EW (total)", "log SFR (component 0)", "log SFR surface density (component 0)"]):
    for ii in range(3):
        cond = df_SF["Number of components"] == ii + 1
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond, col_x].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if ii > 0 else None
axs[0].axvline(np.log10(3), linestyle="--", color="k")
axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"H$\alpha$-derived quantities (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_HaEW_and_sigma_and_SFR_SF_only")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight")

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# Systematic quantities
for cc, col_x in enumerate(["Bin size (square kpc)", "z_spec", "Inclination i (degrees)"]):
    for ii in range(3):
        cond = df_SF["Number of components"] == ii + 1
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_systematic_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# HALPHA S/N
for cc, col_x in enumerate(["HALPHA S/N (component 0)", "HALPHA S/N (component 1)", "HALPHA S/N (component 2)"]):
    for ii in range(3):
        cond = df_SF["Number of components"] == ii + 1
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_HALPHA_SNR_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Specifically looking at v_grad
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["v_grad (component 0)", "v_grad (component 1)", "v_grad (component 2)"]):
    for ii in range(3):
        cond = df_SF["Number of components"] == ii + 1
        if all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", range=(0, 150),
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x) + f" (component {cc + 1})")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$v_{\rm grad}$ (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_vgrad_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Local quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["D4000", "r/R_e", "sigma_*"]):
    for ii in range(3):
        cond = df_SF["Number of components"] == ii + 1
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Local properties (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_local_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Global quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["R_e (kpc)", "log M_*", "log(M/R_e)"]):
    for ii in range(3):
        cond = df_SF["Number of components"] == ii + 1
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Global properties (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_global_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_HaEW_and_sigma_and_SFR_SF_only


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_systematic_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_vgrad_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_local_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_global_SF_only.pdf


### PLOT: histogram showing % of 1,2,3 component spaxels as a function of SFR surface density
---

In [11]:
# Compute: what is the mean SFR surface density in spaxels with 1, 2, 3 components?
print(f"1-component SF spaxels: mean log SFR surface density = {df_SF.loc[df_SF['Number of components'] == 1, 'log SFR surface density (total)'].mean():.4f} ± {df_SF.loc[df_SF['Number of components'] == 1, 'log SFR surface density (total)'].std():.4f}")
print(f"2-component SF spaxels: mean log SFR surface density = {df_SF.loc[df_SF['Number of components'] == 2, 'log SFR surface density (total)'].mean():.4f} ± {df_SF.loc[df_SF['Number of components'] == 2, 'log SFR surface density (total)'].std():.4f}")
print(f"3-component SF spaxels: mean log SFR surface density = {df_SF.loc[df_SF['Number of components'] == 3, 'log SFR surface density (total)'].mean():.4f} ± {df_SF.loc[df_SF['Number of components'] == 3, 'log SFR surface density (total)'].std():.4f}")

print("\n")
print(f"1-component SF spaxels: mean log SFR surface density = {df_SF.loc[df_SF['Number of components'] == 1, 'log SFR surface density (component 0)'].mean():.4f} ± {df_SF.loc[df_SF['Number of components'] == 1, 'log SFR surface density (component 0)'].std():.4f}")
print(f"2-component SF spaxels: mean log SFR surface density = {df_SF.loc[df_SF['Number of components'] == 2, 'log SFR surface density (component 0)'].mean():.4f} ± {df_SF.loc[df_SF['Number of components'] == 2, 'log SFR surface density (component 0)'].std():.4f}")
print(f"3-component SF spaxels: mean log SFR surface density = {df_SF.loc[df_SF['Number of components'] == 3, 'log SFR surface density (component 0)'].mean():.4f} ± {df_SF.loc[df_SF['Number of components'] == 3, 'log SFR surface density (component 0)'].std():.4f}")

NameError: name 'df_SF' is not defined

In [14]:
# Compute how many 1, 2, 3 component spaxels there are in bins of SFR surface density (and SFR)
for col_x in ["log SFR surface density"]:
    sfr_vals = np.linspace(vmin_fn(col_x), vmax_fn(col_x), 10)
    counts_1 = np.zeros(len(sfr_vals) - 1)
    counts_2 = np.zeros(len(sfr_vals) - 1)
    counts_3 = np.zeros(len(sfr_vals) - 1)
    counts_tot = np.zeros(len(sfr_vals) - 1)

    for ll in range(len(sfr_vals) - 1):
        cond = df_SF[f"{col_x} (component 0)"] > sfr_vals[ll]
        cond &= df_SF[f"{col_x} (component 0)"] <= sfr_vals[ll + 1]
        df_subset = df_SF[cond]
        counts_tot[ll] = df_subset.shape[0]
        counts_1[ll] = df_subset[df_subset["Number of components"] == 1].shape[0]
        counts_2[ll] = df_subset[df_subset["Number of components"] == 2].shape[0]
        counts_3[ll] = df_subset[df_subset["Number of components"] == 3].shape[0]

    # Plot
    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 8))
    fig.subplots_adjust(hspace=0)

    # Plot
    axs[0].bar(sfr_vals[:-1], counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
               label="1 component")
    axs[0].bar(sfr_vals[:-1], counts_2, bottom=counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
               label="2 components")
    axs[0].bar(sfr_vals[:-1], counts_3, bottom=counts_1 + counts_2,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
               label="3 components")
    axs[0].grid()
    axs[0].set_ylabel(r"$N$")
    axs[0].set_yscale("log")
    axs[0].autoscale(axis="x", enable=True, tight=True)
    axs[0].set_ylim([0.5, None])
    
    axs[1].bar(sfr_vals[:-1], counts_1 / counts_tot  * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
              label="1 component")
    axs[1].bar(sfr_vals[:-1], counts_2 / counts_tot  * 100, bottom=counts_1 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
              label="2 components")
    axs[1].bar(sfr_vals[:-1], counts_3 / counts_tot * 100, bottom=counts_1 / counts_tot * 100 + counts_2 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
              label="3 components")
    axs[1].grid()
    axs[1].set_ylabel("Percentage")
    axs[1].autoscale(axis="x", enable=True, tight=True)
    axs[1].autoscale(axis="y", enable=True, tight=True)
    axs[1].set_xlabel(label_fn(col_x))

    # Heckman+2002 line
    axs[1].axvline(-1, color="k", linestyle="--", label="Heckman+2002")
    axs[1].axvline(-2, color="k", linestyle=":", label="Roberts-Borsani+2020a")
    axs[1].legend(loc="center left", fontsize="small", bbox_to_anchor=[1.0, 1.0])
        
    if savefigs:
        fname = os.path.join(fig_path, f"hist_SF_only_{col_x.replace(' ', '_')}_ncomponents.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_SF_only_log_SFR_surface_density_ncomponents.pdf


### PLOT: histograms showing local/global/systematic quantities for spaxels with and without at least 1 kinematically disturbed copmonent: what determines the presence of kinematic components with $\Delta \sigma >> 0$?
---

In [15]:
# Halpha EW, SFR, SFR surface density
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["log HALPHA EW (total)", "log SFR (component 0)", "log SFR surface density (component 0)"]):
    for ii in range(3):
        cond = df_SF["Number of kinematically disturbed components"] == ii
        if not df_SF.loc[cond, col_x].any() or all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii  > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond, col_x].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if ii > 0 else None
axs[0].axvline(np.log10(3), linestyle="--", color="k")
axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"H$\alpha$-derived quantities (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_HaEW_and_sigma_and_SFR_SF_only")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight")

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# Systematic quantities
for cc, col_x in enumerate(["Bin size (square kpc)", "z_spec", "Inclination i (degrees)"]):
    for ii in range(3):
        cond = df_SF["Number of kinematically disturbed components"] == ii
        if not df_SF.loc[cond, col_x].any() or all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_systematic_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Specifically looking at v_grad
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["v_grad (component 0)", "v_grad (component 1)", "v_grad (component 2)"]):
    for ii in range(3):
        cond = df_SF["Number of kinematically disturbed components"] == ii
        if not df_SF.loc[cond, col_x].any() or all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", range=(0, 150),
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x) + f" (component {cc + 1})")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$v_{\rm grad}$ (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_vgrad_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Local quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["D4000", "r/R_e", "sigma_*"]):
    for ii in range(3):
        cond = df_SF["Number of kinematically disturbed components"] == ii
        if not df_SF.loc[cond, col_x].any() or all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Local properties (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_local_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Global quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["R_e (kpc)", "log M_*", "log(M/R_e)"]):
    for ii in range(3):
        cond = df_SF["Number of kinematically disturbed components"] == ii
        if not df_SF.loc[cond, col_x].any() or all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii  > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Global properties (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_global_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_HaEW_and_sigma_and_SFR_SF_only


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_systematic_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_vgrad_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_local_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_global_SF_only.pdf


In [16]:
# Compute how many spaxels w/ KD components there are in bins of SFR surface density (and SFR)
for col_x in ["log SFR surface density"]:
    sfr_vals = np.linspace(vmin_fn(col_x), vmax_fn(col_x), 10)
    counts_1 = np.zeros(len(sfr_vals) - 1)
    counts_2 = np.zeros(len(sfr_vals) - 1)
    counts_3 = np.zeros(len(sfr_vals) - 1)
    counts_tot = np.zeros(len(sfr_vals) - 1)

    for ll in range(len(sfr_vals) - 1):
        cond = df_SF[f"{col_x} (component 0)"] > sfr_vals[ll]
        cond &= df_SF[f"{col_x} (component 0)"] <= sfr_vals[ll + 1]
        df_subset = df_SF[cond]
        counts_tot[ll] = df_subset.shape[0]
        counts_1[ll] = df_subset[df_subset["Number of kinematically disturbed components"] == 0].shape[0]
        counts_2[ll] = df_subset[df_subset["Number of kinematically disturbed components"] == 1].shape[0]
        counts_3[ll] = df_subset[df_subset["Number of kinematically disturbed components"] == 2].shape[0]

    # Plot
    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 8))
    fig.subplots_adjust(hspace=0)

    # Plot
    axs[0].bar(sfr_vals[:-1], counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1])
    axs[0].bar(sfr_vals[:-1], counts_2, bottom=counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2])
    axs[0].bar(sfr_vals[:-1], counts_3, bottom=counts_1 + counts_2,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3])
    axs[0].grid()
    axs[0].set_ylabel(r"$N$")
    axs[0].set_yscale("log")
    axs[0].autoscale(axis="x", enable=True, tight=True)
    axs[0].set_ylim([0.5, None])
    
    axs[1].bar(sfr_vals[:-1], counts_1 / counts_tot  * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
               label="0 KD components")
    axs[1].bar(sfr_vals[:-1], counts_2 / counts_tot  * 100, bottom=counts_1 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
               label="1 KD component")
    axs[1].bar(sfr_vals[:-1], counts_3 / counts_tot * 100, bottom=counts_1 / counts_tot * 100 + counts_2 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
               label="2 KD components")
    axs[1].grid()
    axs[1].set_ylabel("Percentage")
    axs[1].autoscale(axis="x", enable=True, tight=True)
    axs[1].autoscale(axis="y", enable=True, tight=True)
    axs[1].set_xlabel(label_fn(col_x))

    # Heckman+2002 line
    axs[1].axvline(-1, color="k", linestyle="--", label="Heckman+2002")
    axs[1].axvline(-2, color="k", linestyle=":", label="Roberts-Borsani+2020a")
    axs[1].legend(loc="center left", fontsize="small", bbox_to_anchor=[1.0, 1.0])
        
    if savefigs:
        fname = os.path.join(fig_path, f"hist_SF_only_{col_x.replace(' ', '_')}_KDcomponents.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_SF_only_log_SFR_surface_density_KDcomponents.pdf


### ~PLOT: WHAV* diagrams for SF spaxels in slices of SFR surface density and SFR~
---

In [25]:
col_z = "count"
sfr_bounds = [
    (-4, -3),
    (-3, -2),
    (-2, -1),
    (-1, 0)
]
for sfr_min, sfr_max in sfr_bounds:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    df_slice = df[(df["log SFR surface density (component 0)"] > sfr_min) & (df["log SFR surface density (component 0)"] <= sfr_max)]
    
    for ii in range(3):
        plot2dhistcontours(df_slice, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title(r"$%.1f < \log_{10} \Sigma_{\rm SFR} \leq %.1f$" % (sfr_min, sfr_max))

    # Save
#     if savefigs:
#         fname = os.path.join(fig_path, f"WHAV_SAMI_{fname_fn(col_z)}.pdf")
#         fig.savefig(fname, bbox_inches="tight", format="pdf")
#         print(f"File saved at: {fname}")      


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:178: UserWarning: No contour levels were found within the data range.
  linewidths=linewidths)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PLOT: WHAV* diagrams for SF spaxels in slices of *total* galaxy SFR
---

In [26]:
# Compute total galaxy SFR 
for gal in df["ID"].unique():
    df_gal = df[df["ID"] == gal]
    SFR_tot_comp0 = df_gal["SFR (component 0)"].sum()
    SFR_tot = df_gal["SFR (total)"].sum()
    mstar = df_gal["log M_*"].unique()[0]
    df.loc[df["ID"] == gal, "SFR (whole galaxy, component 0)"] = SFR_tot_comp0
    df.loc[df["ID"] == gal, "SFR (whole galaxy, total)"] = SFR_tot
    df.loc[df["ID"] == gal, "sSFR (whole galaxy, component 0)"] = SFR_tot_comp0 / 10**mstar
    df.loc[df["ID"] == gal, "sSFR (whole galaxy, total)"] = SFR_tot / 10**mstar
    
df["log SFR (whole galaxy, component 0)"] = np.log10(df["SFR (whole galaxy, component 0)"])
df["log SFR (whole galaxy, total)"] = np.log10(df["SFR (whole galaxy, total)"])
df["log sSFR (whole galaxy, component 0)"] = np.log10(df["sSFR (whole galaxy, component 0)"])
df["log sSFR (whole galaxy, total)"] = np.log10(df["sSFR (whole galaxy, total)"])

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [27]:
plt.figure(); plt.hist(df["log sSFR (whole galaxy, total)"].values, range=(-18, -7))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:829: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:830: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([     0.,      0.,  41084., 171651., 183161., 149773., 474796.,
        440680.,   5456.,      0.]),
 array([-18. , -16.9, -15.8, -14.7, -13.6, -12.5, -11.4, -10.3,  -9.2,
         -8.1,  -7. ]),
 <a list of 10 Patch objects>)

In [28]:
# Slices of overall SFR 
col_z = "count"
sfr_bounds = [
    (-5, -4),
    (-4, -3),
    (-3, -2),
    (-2, -1),
    (-1, 0),
    (0, 1)
]
for sfr_min, sfr_max in sfr_bounds:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    df_slice = df[(df["log SFR (whole galaxy, component 0)"] > sfr_min) & (df["log SFR (whole galaxy, component 0)"] <= sfr_max)]
    
    for ii in range(3):
        plot2dhistcontours(df_slice, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title(r"$%.1f < \log_{10} {\rm SFR_{tot}} \leq %.1f$" % (sfr_min, sfr_max))

    # Save
#     if savefigs:
#         fname = os.path.join(fig_path, f"WHAV_SAMI_{fname_fn(col_z)}.pdf")
#         fig.savefig(fname, bbox_inches="tight", format="pdf")
#         print(f"File saved at: {fname}")      


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
# Slices of specific SFR 
col_z = "count"
ssfr_bounds = [
    (-16, -14),
    (-14, -12),
    (-12, -10),
    (-10, -8),
]
for ssfr_min, ssfr_max in ssfr_bounds:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    df_slice = df[(df["log sSFR (whole galaxy, component 0)"] > ssfr_min) & (df["log sSFR (whole galaxy, component 0)"] <= ssfr_max)]
    
    for ii in range(3):
        plot2dhistcontours(df_slice, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title(r"$%.1f < \log_{10} {\rm sSFR_{tot}} \leq %.1f$" % (ssfr_min, ssfr_max))

    # Save
#     if savefigs:
#         fname = os.path.join(fig_path, f"WHAV_SAMI_{fname_fn(col_z)}.pdf")
#         fig.savefig(fname, bbox_inches="tight", format="pdf")
#         print(f"File saved at: {fname}")      


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute: fraction of spaxels with $\Delta\sigma < 0$ in the multi-component fits and in the single-component fits
---

In [30]:
# How many SF-like spaxels have dsigma < 0?
Ntot = df_SF[(df_SF["Number of components"] == 1) & (~df_SF["sigma_gas - sigma_* (component 0)"].isna())].shape[0]
N = df_SF[(df_SF["Number of components"] == 1) & (df_SF["sigma_gas - sigma_* (component 0)"] < 0)].shape[0]
N2 = df_SF[(df_SF["Number of components"] == 1) & (df_SF["sigma_gas - sigma_* (component 0)"] > 0)].shape[0]

print(f"\n{N / Ntot * 100}% of 1-component SF-like spaxels have component 0 with dsigma < 0")
print(f"{N2 / Ntot * 100}% of 1-component SF-like spaxels have component 0 with dsigma > 0")
print(f"The median value of dsigma in these spaxels is {df_SF.loc[(df_SF['Number of components'] == 1), 'sigma_gas - sigma_* (component 0)'].mean()} km/s")
print(f"The standard deviation in dsigma in these spaxels is {df_SF.loc[(df_SF['Number of components'] == 1), 'sigma_gas - sigma_* (component 0)'].std()} km/s")

Ntot = df_SF[(df_SF["Number of components"] == 2) & (~df_SF["sigma_gas - sigma_* (component 0)"].isna())].shape[0]
N = df_SF[(df_SF["Number of components"] == 2) & (df_SF["sigma_gas - sigma_* (component 0)"] < 0)].shape[0]
N2 = df_SF[(df_SF["Number of components"] == 2) & (df_SF["sigma_gas - sigma_* (component 0)"] > 0)].shape[0]

print(f"\n{N / Ntot * 100}% of 2-component SF-like spaxels have component 0 with dsigma < 0")
print(f"{N2 / Ntot * 100}% of 2-component SF-like spaxels have component 0 with dsigma > 0")
print(f"The median value of dsigma in these spaxels is {df_SF.loc[(df_SF['Number of components'] == 2), 'sigma_gas - sigma_* (component 0)'].mean()} km/s")
print(f"The standard deviation in dsigma in these spaxels is {df_SF.loc[(df_SF['Number of components'] == 2), 'sigma_gas - sigma_* (component 0)'].std()} km/s")

Ntot = df_SF[(df_SF["Number of components"] == 3) & (~df_SF["sigma_gas - sigma_* (component 0)"].isna())].shape[0]
N = df_SF[(df_SF["Number of components"] == 3) & (df_SF["sigma_gas - sigma_* (component 0)"] < 0)].shape[0]
N2 = df_SF[(df_SF["Number of components"] == 3) & (df_SF["sigma_gas - sigma_* (component 0)"] > 0)].shape[0]

print(f"\n{N / Ntot * 100}% of 3-component SF-like spaxels have component 0 with dsigma < 0")
print(f"{N2 / Ntot * 100}% of 3-component SF-like spaxels have component 0 with dsigma > 0")
print(f"The median value of dsigma in these spaxels is {df_SF.loc[(df_SF['Number of components'] == 3), 'sigma_gas - sigma_* (component 0)'].mean()} km/s")
print(f"The standard deviation in dsigma in these spaxels is {df_SF.loc[(df_SF['Number of components'] == 3), 'sigma_gas - sigma_* (component 0)'].std()} km/s")

print(f"\nThe median value of dsigma in all SF-like spaxels is {df_SF['sigma_gas - sigma_* (component 0)'].mean()} km/s")
print(f"The standard deviation in dsigma in all SF-like spaxels is {df_SF['sigma_gas - sigma_* (component 0)'].std()} km/s")
print(f"\nThe median value of sigma_gas in all SF-like spaxels is {df_SF['sigma_gas (component 0)'].mean()} km/s")
print(f"The standard deviation in sigma_gas in all SF-like spaxels is {df_SF['sigma_gas (component 0)'].std()} km/s")



97.8871396280287% of 1-component SF-like spaxels have component 0 with dsigma < 0
2.1128603719712893% of 1-component SF-like spaxels have component 0 with dsigma > 0
The median value of dsigma in these spaxels is -38.41897190193553 km/s
The standard deviation in dsigma in these spaxels is 26.5349240691652 km/s

99.48364888123923% of 2-component SF-like spaxels have component 0 with dsigma < 0
0.5163511187607573% of 2-component SF-like spaxels have component 0 with dsigma > 0
The median value of dsigma in these spaxels is -48.353959925477085 km/s
The standard deviation in dsigma in these spaxels is 26.76919846461757 km/s

99.64945080626315% of 3-component SF-like spaxels have component 0 with dsigma < 0
0.35054919373685445% of 3-component SF-like spaxels have component 0 with dsigma > 0
The median value of dsigma in these spaxels is -47.72063281115906 km/s
The standard deviation in dsigma in these spaxels is 27.222795238463696 km/s

The median value of dsigma in all SF-like spaxels is 

### PLOT: WHAV* diagrams for SF spaxels coloured by SFR and SFR surface density
---

In [17]:
col_z_list = ["log SFR surface density (component 0)"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           cmap="coolwarm_r", vmin=-3, vmax=-1,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        
col_z_list = ["log SFR (component 0)"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=-4, vmax=-2,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
      

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_logSFR_surface_density.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_logSFR.pdf


In [32]:
# Coloured by sigma_gas, sigma_*
col_z_list = ["sigma_gas", "sigma_*"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5, 
                           vmax=200, cmap="Spectral_r",
                           contours=True, hist=True, colors="white",
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_sigma_gas.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_sigma_star.pdf


### PLOT: EW vs. $v_{\rm gas}$: is the second component blueshifted at all?
---

In [33]:
df_SF["delta v_gas (2/0)"] = df_SF["v_gas (component 2)"] - df_SF["v_gas (component 0)"]
df_SF["delta v_gas (2/0)"].mean()

-16.70810163215178

In [34]:
# Histogram showing distribution in v_gas (component 2) - v_gas (component 1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5))
hist(df_SF["delta v_gas (1/0)"], range=(-250, +250), bins="scott", histtype="step", ax=ax, label=label_fn("delta v_gas (1/0)"))
hist(df_SF["delta v_gas (2/1)"], range=(-250, +250), bins="scott", histtype="step", ax=ax, label=label_fn("delta v_gas (2/1)"))
hist(df_SF["delta v_gas (2/0)"], range=(-250, +250), bins="scott", histtype="step", ax=ax, label=label_fn("delta v_gas (2/0)"))
ax.axvline(0, color="k")
ax.set_xlabel(r"$\Delta v_{\rm gas}$")
ax.set_ylabel(r"$N$")
ax.legend(loc="upper right")
ax.grid()

if savefigs:
    fname = os.path.join(fig_path, "hist_delta_v_gas.pdf")
    print(f"Saving file to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving file to /priv/meggs3/u5708159/SAMI/figs/paper/hist_delta_v_gas.pdf


In [35]:
df_SF["delta sigma_gas (1/0)"].std()

49.12195238618891

In [36]:
df_SF["delta sigma_gas (2/0)"] = df_SF["sigma_gas (component 2)"] - df_SF["sigma_gas (component 0)"]
df_SF["delta sigma_gas (2/0)"].mean()

# Histogram showing distribution in sigma_gas (component 2) - sigma_gas (component 1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5))
hist(df_SF["delta sigma_gas (1/0)"], range=(0, +400), bins="scott", histtype="step", ax=ax, label=label_fn("delta sigma_gas (1/0)"))
hist(df_SF["delta sigma_gas (2/1)"], range=(0, +400), bins="scott", histtype="step", ax=ax, label=label_fn("delta sigma_gas (2/1)"))
hist(df_SF["delta sigma_gas (2/0)"], range=(0, +400), bins="scott", histtype="step", ax=ax, label=label_fn("delta sigma_gas (2/0)"))
ax.set_xlabel(r"$\Delta \sigma_{\rm gas}$")
ax.set_ylabel(r"$N$")
ax.legend(loc="upper right")
ax.grid()
ax.autoscale(enable=True, axis="x", tight=True)

if savefigs:
    fname = os.path.join(fig_path, "hist_delta_sigma_gas.pdf")
    print(f"Saving file to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving file to /priv/meggs3/u5708159/SAMI/figs/paper/hist_delta_sigma_gas.pdf


In [37]:
# Histogram showing distribution in v_gas - v_* in slices of SFR surface density 


In [38]:
# Plot showing SFR surface density vs. v_gas (component 2) - v_gas (component 1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
plot2dhistcontours(df_SF, 
                   col_x="delta v_gas (1/0)",
                   col_y="log SFR surface density (component 0)",
                   col_z="count", log_z=True,
                   alpha=1.0, ax=ax, nbins=100,
                   linewidths=0.5, 
                   xmin=-400, xmax=+400,
                   contours=True, hist=True, colors="white",
                   plot_colorbar=True)
ax.grid()
ax.axvline(0, color="k")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
# Coloured by sigma_gas, sigma_*
col_z_list = ["v_gas - v_*"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, 
                           col_x=f"v_gas - v_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5, 
                           xmin=-400, xmax=+400,
                           vmin=-200, vmax=+200,
                           cmap="Spectral_r",
                           contours=True, hist=True, colors="white",
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].axvline(0, color="k")
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_v_gas-v_star.pdf


In [40]:
plt.close("all")

## PLOT: histogram showing $\sigma_{\rm gas}$ and $\sigma_*$ vs. $\Delta\sigma$
---

In [41]:
# Coloured by sigma_gas, sigma_*
col_z_list = ["count"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, 
                           col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"sigma_gas (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5, ymax=500,
                           contours=True, hist=True, colors="white",
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"sigma_gas_vs_dsigma_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, 
                           col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"sigma_*",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5, 
                           contours=True, hist=True, colors="white",
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"sigma_star_vs_dsigma_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/sigma_gas_vs_dsigma_SAMI_SF_only_count.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/sigma_star_vs_dsigma_SAMI_SF_only_count.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


## PLOT: WHAV* diagram for the whole sample coloured by count, BPT category
---

In [42]:
col_z_list = ["count", "BPT (numeric) (total)"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_{fname_fn(col_z)}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")        
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_count.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_BPT.pdf


### ~PLOT: histograms of $\Delta \sigma$ in slices of SFR surface density to see if there is a correlation~
---

In [43]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
fig.subplots_adjust(wspace=0)

sfr_bounds = [
    (-5, -3),
    (-3, -1),
    (-1, 1)
]
sfr_colours = [
    "darkred", "red", "orange"
]
for (sfr_min, sfr_max), colour in zip(sfr_bounds, sfr_colours):
    df_slice = df[(df["log SFR surface density (component 0)"] > sfr_min) & (df["log SFR surface density (component 0)"] <= sfr_max)]
    
    for ii in range(3):
        hist(df_slice[f"sigma_gas - sigma_* (component {ii})"], ax=axs[ii], 
             color=colour,
             range=(vmin_fn("sigma_gas - sigma_*"), vmax_fn("sigma_gas - sigma_*")), 
             bins="scott", histtype="step", normed=True)
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


### PLOT: WHAV* diagrams for SF-only spaxels coloured by various properties to see what drives the spread in the diagram
---

In [44]:
plt.close("all")

In [8]:
col_z_list = ["count", "log M_*", "r/R_e", "sigma_*", "log HALPHA continuum luminosity", "log HALPHA luminosity"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           xmax=600,
                           contours=True, hist=True, colors="white",
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_SF_only_{fname_fn(col_z)}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_count.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_mstar.pdf


...

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_sigma_star.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_log_HALPHA_cont_lum_per_kpc2.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_SF_only_log_HALPHA_lum_per_kpc2.pdf


# **Analysis of AGN-like spaxels**
---
---

In [46]:
# Need to correct [OIII] for extinction because it's such a blue line
import extinction

def get_A_V(ratio):
    
    # Calculate A(Ha), A(Hb) from emission lines
    intrinsic_ratio = 2.85
    E_ba = 2.5 * (np.log10(ratio)) - 2.5 * np.log10(intrinsic_ratio)

    # Calculate ( A(Ha) - A(Hb) ) / E(B-V) from extinction curve
    R_V = 3.1
    wave_1_A = np.array([6562.800])
    wave_2_A = np.array([4861.325])

    # A_V is a multiplicative scale factor for the extinction curve. So the below calculation is the same regardless of A_V because we normalise by it.
    E_ba_over_E_BV = (extinction.fm07(wave_2_A,a_v=1.0)[0] - extinction.fm07(wave_1_A,a_v=1.0)[0] ) /  1.0 * R_V

    # Calculate E(B-V)
    E_BV = 1 / E_ba_over_E_BV * E_ba

    # Calculate A(V)
    A_V = R_V * E_BV

    return A_V

def ext_corr_factors(A_V_vals, lambda_A):
    A_line_vals = []
    for A_V in A_V_vals:
        A_line = extinction.fm07(np.array([lambda_A]), a_v=A_V)[0]
        A_line_vals.append(A_line)
    return 10**(0.4 * np.array(A_line_vals))


In [47]:
# Compute the total [OIII] luminosity for each galaxy
gals = df["ID"].unique()
for gal in tqdm(gals):

    df_gal = df[df["ID"] == gal]
    agn_cond = ((df_gal["BPT (total)"] == "Seyfert") | (df_gal["BPT (total)"] == "LINER")) & (df_gal["HALPHA EW (total)"] > 3)
    if any(agn_cond):
        
        # Determine the extinction correction factor for [OIII] for these spaxels only
        LOIII_vals = df_gal.loc[agn_cond, "OIII5007 (total)"].values * 1e-16 * 4 * np.pi * (df_gal["D_L (Mpc)"].unique()[0] * 1e6 * 3.086e18)**2  # erg s^-1
        Ha_Hb_ratio_vals = df_gal.loc[agn_cond, "HALPHA (total)"].values / df_gal.loc[agn_cond, "HBETA (total)"].values
        Ha_Hb_ratio_err_vals = Ha_Hb_ratio_vals * np.sqrt((df_gal.loc[agn_cond, "HALPHA error (total)"].values / df_gal.loc[agn_cond, "HALPHA (total)"].values)**2 +\
                                                          (df_gal.loc[agn_cond, "HBETA error (total)"].values / df_gal.loc[agn_cond, "HBETA (total)"].values)**2)
        Ha_Hb_SNR_vals = Ha_Hb_ratio_vals / Ha_Hb_ratio_err_vals
        Ha_Hb_ratio_vals[Ha_Hb_SNR_vals < 3] = 2.85  # Do not perform extinction correction if the S/N in the ratio is poor
        A_V_vals = get_A_V(Ha_Hb_ratio_vals)
        OIII_ext_corr_vals = ext_corr_factors(A_V_vals, 5007)
        LOIII_ext_corr_vals = LOIII_vals * OIII_ext_corr_vals
        LOIII_ext_corr_tot = np.nansum(LOIII_ext_corr_vals)
        LOIII_tot = np.nansum(LOIII_vals)
        
        # Store in DataFrame
        df.loc[df["ID"] == gal, "log [OIII] luminosity (AGN, total)"] = np.log10(LOIII_tot)
        df.loc[df["ID"] == gal, "log [OIII] luminosity (AGN, ext. corr., total)"] = np.log10(LOIII_ext_corr_tot)


100%|██████████| 3005/3005 [00:14<00:00, 205.12it/s]


In [55]:
plt.figure()
plt.hist(df["log [OIII] luminosity (AGN, total)"], range=(38, 42), histtype="step", bins=25)
plt.hist(df["log [OIII] luminosity (AGN, ext. corr., total)"], range=(38, 42), histtype="step", bins=25)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:829: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:830: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([ 4344.,  2644.,  3837.,  2640., 12556.,  3592.,  3410.,  5473.,
         2875.,  3527., 13227.,  4188.,  4785.,  4246.,  7005.,  5848.,
         6785.,  2305.,  3787.,  4594.,   667.,  2488.,  2788.,   648.,
         2114.]),
 array([38.  , 38.16, 38.32, 38.48, 38.64, 38.8 , 38.96, 39.12, 39.28,
        39.44, 39.6 , 39.76, 39.92, 40.08, 40.24, 40.4 , 40.56, 40.72,
        40.88, 41.04, 41.2 , 41.36, 41.52, 41.68, 41.84, 42.  ]),
 <a list of 1 Patch objects>)

In [56]:
# Make a separate DataFrame only containing star-forming spaxels
df_AGN = df.copy()
df_AGN = df_AGN[((df_AGN["BPT (total)"] == "Seyfert") | (df_AGN["BPT (total)"] == "LINER")) & (df_AGN["HALPHA EW (total)"] > 3)]


In [57]:
df_AGN["log [OIII] luminosity (AGN, ext. corr., total)"].values

array([39.75695599, 40.99042072, 40.99042072, ..., 40.57058546,
       40.57058546, 40.57058546])

### PLOT: histogram for AGN-only spaxels showing % of 1,2,3 component spaxels as a function of local, global and systematic properties
---

In [58]:
# Halpha EW, SFR, SFR surface density
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["log HALPHA EW (total)", "log HALPHA luminosity (total)"]):
    for ii in range(3):
        cond = df_AGN["Number of components"] == ii + 1
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond, col_x].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if ii > 0 else None
axs[0].axvline(np.log10(3), linestyle="--", color="k")
axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"H$\alpha$-derived quantities (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_HaEW_and_sigma_and_SFR_AGN_only")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight")

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# Systematic quantities
for cc, col_x in enumerate(["Bin size (square kpc)", "z_spec", "Inclination i (degrees)"]):
    for ii in range(3):
        cond = df_AGN["Number of components"] == ii + 1
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_systematic_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Specifically looking at v_grad
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["v_grad (component 0)", "v_grad (component 1)", "v_grad (component 2)"]):
    for ii in range(3):
        cond = df_AGN["Number of components"] == ii + 1
        if all(df_AGN.loc[cond, col_x].isna()):
            continue
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", range=(0, 150),
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x) + f" (component {cc + 1})")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$v_{\rm grad}$ (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_vgrad_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Local quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["D4000", "r/R_e", "sigma_*"]):
    for ii in range(3):
        cond = df_AGN["Number of components"] == ii + 1
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Local properties (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_local_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Global quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["R_e (kpc)", "log M_*", "log(M/R_e)"]):
    for ii in range(3):
        cond = df_AGN["Number of components"] == ii + 1
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             range=(vmin_fn(col_x), vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Global properties (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_global_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Total OIII luminosity
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
fig.subplots_adjust(wspace=0)
axs = [ax]
for cc, col_x in enumerate(["log [OIII] luminosity (AGN, ext. corr., total)"]):
    for ii in range(3):
        cond = df_AGN["Number of components"] == ii + 1
        hist(df_AGN.loc[cond, col_x] + np.log10(3500), density=True, histtype="step",
             ax=axs[cc],
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(r"$\log_{10} (3500 L(\rm [O\,III]) \, [\rm erg\,s{^-1}])$")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$L_{\rm Edd}$ (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_LOIII_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_HaEW_and_sigma_and_SFR_AGN_only


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_systematic_AGN_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_local_AGN_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_global_AGN_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_1vs2vs3components_LOIII_AGN_only.pdf


### PLOT: histograms showing local/global/systematic quantities for spaxels with and without at least 1 kinematically disturbed copmonent: what determines the presence of kinematic components with $\Delta \sigma >> 0$?
---

In [59]:
# New columns: total number of kinematically disturbed components
df_AGN["Number of kinematically disturbed components"] = df_AGN["Kinematically disturbed (component 0)"].astype(int) + df_AGN["Kinematically disturbed (component 1)"].astype(int) + df_AGN["Kinematically disturbed (component 2)"].astype(int)


In [60]:
# Halpha EW, SFR, SFR surface density
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["log HALPHA EW (total)", "log HALPHA luminosity (total)"]):
    for ii in range(3):
        cond = df_AGN["Number of kinematically disturbed components"] == ii
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii  > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond, col_x].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if ii > 0 else None
axs[0].axvline(np.log10(3), linestyle="--", color="k")
axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"H$\alpha$-derived quantities (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_HaEW_and_sigma_and_SFR_AGN_only")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight")

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# Systematic quantities
for cc, col_x in enumerate(["Bin size (square kpc)", "z_spec", "Inclination i (degrees)"]):
    for ii in range(3):
        cond = df_AGN["Number of kinematically disturbed components"] == ii
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_systematic_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Specifically looking at v_grad
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["v_grad (component 0)", "v_grad (component 1)", "v_grad (component 2)"]):
    for ii in range(3):
        cond = df_AGN["Number of kinematically disturbed components"] == ii
        if all(df_AGN.loc[cond, col_x].isna()):
            continue
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", range=(0, 150),
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x) + f" (component {cc + 1})")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$v_{\rm grad}$ (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_vgrad_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Local quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["D4000", "r/R_e", "sigma_*"]):
    for ii in range(3):
        cond = df_AGN["Number of kinematically disturbed components"] == ii
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Local properties (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_local_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Global quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["R_e (kpc)", "log M_*", "log(M/R_e)"]):
    for ii in range(3):
        cond = df_AGN["Number of kinematically disturbed components"] == ii
        hist(df_AGN.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii  > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Global properties (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_global_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Total OIII luminosity
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
fig.subplots_adjust(wspace=0)
axs = [ax]
for cc, col_x in enumerate(["log [OIII] luminosity (AGN, ext. corr., total)"]):
    for ii in range(3):
        cond = df_AGN["Number of kinematically disturbed components"] == ii
        hist(df_AGN.loc[cond, col_x] + np.log10(3500), density=True, histtype="step",
             ax=axs[cc],
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_AGN.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(r"$\log_{10} (3500 L(\rm [O\,III]) \, [\rm erg\,s{^-1}])$")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$L_{\rm Edd}$ (AGN spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KD_LOIII_AGN_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_HaEW_and_sigma_and_SFR_AGN_only


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_systematic_AGN_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_local_AGN_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KDcomponents_global_AGN_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/hist_KD_LOIII_AGN_only.pdf


### PLOT: WHAV* diagrams for AGN spaxels in slices of total galaxy [O III] luminosity
---

In [61]:
# What are these galaxies that have multiple AGN-like kinematic components but low [OIII] luminosities?
cond = df_AGN["log [OIII] luminosity (AGN, ext. corr., total)"] <= 40
cond &= df_AGN["sigma_gas - sigma_* (component 1)"] > 0
cond &= df_AGN["log HALPHA EW (component 1)"] > 0.5
df_AGN[cond]["ID"]

2093098    9011900521
350204         593680
350604         593680
657029          48016
825363         592421
825364         592421
1179185        618992
1252418        537426
1252419        537426
1252468        537426
1252469        537426
2092859    9011900521
2093056    9011900521
2093066    9011900521
2093089    9011900521
2093093    9011900521
2093094    9011900521
2093124    9011900521
2093126    9011900521
2093128    9011900521
2093157    9011900521
588151         348116
667729         202531
667757         202531
2192399    9016800276
2192460    9016800276
2192629    9016800276
2192660    9016800276
2192689    9016800276
2192690    9016800276
Name: catid, dtype: int64

In [62]:
col_z = "BPT (numeric) (total)"
LOIII_bounds = [
    (39, 40),
    (40, 41),
    (41, 42),
    (42, np.inf)
]
for LOIII_min, LOIII_max in LOIII_bounds:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    df_slice = df_AGN[(df_AGN["log [OIII] luminosity (AGN, ext. corr., total)"] > LOIII_min) & (df_AGN["log [OIII] luminosity (AGN, ext. corr., total)"] <= LOIII_max)]
    
    for ii in range(3):
        plot2dhistcontours(df_slice, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_AGN else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title(r"$%.1f < \log_{10} L(\rm [O\,III]) \leq %.1f$" % (LOIII_min, LOIII_max))

    # Save
#     if savefigs:
#         fname = os.path.join(fig_path, f"WHAV_SAMI_{fname_fn(col_z)}.pdf")
#         fig.savefig(fname, bbox_inches="tight", format="pdf")
#         print(f"File saved at: {fname}")      


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:178: UserWarning: No contour levels were found within the data range.
  linewidths=linewidths)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PLOT: WHAV* diagrams for AGN-only spaxels coloured by various properties to see what drives the spread in the diagram
---

In [63]:
col_z_list = ["log [OIII] luminosity (AGN, ext. corr., total)"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_AGN, col_x=f"sigma_gas - sigma_* (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_AGN else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           vmin=39, vmax=42, cmap="inferno",
                           contours=True, hist=True, colors="white",
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
        axs[1].set_title("SF spaxels only")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_SAMI_AGN_only_LOIII.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/paper/WHAV_SAMI_AGN_only_LOIII.pdf
